<a href="https://colab.research.google.com/github/Valeeew/Data_management_of_an_online_store/blob/master/Data_management_of_an_online_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Management & Optimisation d'une boutique en ligne
## Par Valentin MONTEIRO 
---
### Sommaire
> ### IMPORT
- [Import des library](#scrollTo=tAgUgPb2dUt3&line=1&uniqifier=1)
- [Import des DATA](#scrollTo=LGGkJyQgd2Vc&line=3&uniqifier=1)
- [Traitement des DATA](#scrollTo=THwlX1u91f2t&line=4&uniqifier=1)  
- [Lexique des données](#scrollTo=oG89DAzeosCW&line=1&uniqifier=1)

> ### Mission
- [Mission](#scrollTo=zVQGfyVrRI86&line=15&uniqifier=1)



## IMPORT

---
> ### Import des Libraries
---



In [1]:
import pandas as pd

---
> ### Import des DATA from Github
---




In [2]:
# Clone the entire repo & import DATA
!git clone -l -s git://github.com/Valeeew/Data_management_of_an_online_store cloned-repo
%cd cloned-repo
!ls

# Enregistrement des DATA dans les DATAframe
df_erp = pd.read_excel("erp.xlsx")
df_liaison = pd.read_excel("liaison.xlsx")
df_web = pd.read_excel("web.xlsx")

fatal: destination path 'cloned-repo' already exists and is not an empty directory.
/content/cloned-repo
cloned-repo				  erp.xlsx	web.xlsx
Data_management_of_an_online_store.ipynb  liaison.xlsx


---
> ### Traitement des DATA
---




In [3]:
# Traitement des données web inutiles
df_web = df_web.drop(["virtual","downloadable","rating_count","average_rating","tax_class","post_content","comment_status","ping_status","post_password","post_content_filtered","post_parent","menu_order","comment_count"], axis = 1)
# Remplacement des données NaN dans df_web.total_sales
df_web.total_sales = df_web.total_sales.fillna(0)
# Remplacement des données tax_status ➡️ taxable = 1 et Nan = 0
df_web.tax_status = df_web.tax_status.fillna(0)
df_web.tax_status = df_web.tax_status.replace("taxable",1)

---
> ### Lexique des données
---





#### df_erp

| Variable | Description | Type | PK |
| :--------------- |:---------------| :-----| :---|
| product_id | références produit | int | ✅ |
| onsale_web | vendu en ligne | boolean ||
| price | prix de vente | float ||
| stock_quantity | quantité dans le stock | int ||
|stock_status | État de stock | str ||

#### df_web

| Variable | Description | Type | PK |
| :--------------- |:---------------| :-----| :---|
| sku | références produit | int | ✅ |
| total_sales | nombre de vente | Int ||
| tax_status | produit taxable ou non | Boolean ||
| post_author | auteur de l'article | int ||
| post_date | Date de publication | date ||
| post_date_gmt | Date de publication h GMT| date ||
| post_title | titre de l'article | str ||
| post_excerpt | description de l'article | str ||
| post_status | statut de l'article | str ||
| post_name | Nom de l'article | str ||
| post_modified | Date dernière MAJ | date ||
| post_modified_gmt | Date dernière MAJ h GMT | date ||
| guid | lien | str ||
| post_type | type de l'article | str ||
| post_mime_type | Chemin de l'image | str ||

---
> ### Mission
---

Ta première mission se passe en 3 points.

Premièrement, j’ai besoin que tu rapproches deux exports : un export de l’ERP et un export d’une table de l’outil de CMS contenant les informations des produits commercialisés en ligne (nom, description, nombre de ventes...).

L’export issu de la boutique en ligne contient le nombre de ventes pour chaque produit depuis sa mise en ligne, il ne permet pas d’analyser l'évolution des ventes dans le temps.

Je vais t’envoyer un mail dès la fin de notre entretien avec ces 2 exports en pièce jointe.

En plus de ces 2 exports, tu vas bénéficier d’une aide précieuse car Sylvie, notre ancienne stagiaire, a réalisé un travail de fourmi. Elle a créé un tableau Excel qui permet d’établir le lien entre la référence du produit dans l’ERP (product_id) et la référence du même produit dans la base de la boutique en ligne (SKU). 

Deuxièmement, une fois le rapprochement effectué, je souhaiterais avoir le chiffre d’affaires par produit, ainsi que le total du chiffre d’affaires réalisé en ligne.

Troisièmement et pour finir, je me demande s’il n’y a pas eu des erreurs de saisie dans certains prix des produits. J'aimerais que tu effectues une analyse sur cette variable afin de détecter d’éventuelles valeurs aberrantes, de les lister et d’en faire une représentation graphique pour plus de lisibilité.

Nous voudrions présenter tes résultats lors de la prochaine réunion de COPIL. Cela permettrait de montrer nos progrès. Je peux t’épargner le travail d’une présentation par slides, il me faut juste un notebook que tu présenteras à l’assemblée pour expliquer ta démarche. Tu peux utiliser R ou Python comme tu préfères, nous n'avons pas de préférence de notre côté.

---
> ### Calcul
---